In [ ]:
import warnings
warnings.filterwarnings("ignore")
from MT5 import *
import numpy as np
import ta
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
import time
import pickle
from joblib import dump, load
import os

path = "" # Ex: C:/Desktop/Python_for_finance_and_algorithmic_trading/ChapterN/Models


def create_model_weights(symbol):
    """ Weights for Linear regression on the percentage change"""
    # Import the data
    data = MT5.get_data(symbol, 3500)[["close"]].pct_change(1)
    
    # Create new variable
    data["close t-1"] = data[["close"]].shift(1)
    
    # Split the data
    data = data.dropna()
    split = int(0.80*len(data))
    
    # Train set creation
    X_train = data[["close t-1"]].iloc[:split]
    y_train = data[["close"]].iloc[:split]

    # Test set creation
    X_test = data[["close t-1"]].iloc[split:]
    
    # Create the model
    alg = LinearRegression()

    # Fit the model
    alg.fit(X_train, y_train)
    
    # Save the model
    alg_var = pickle.dumps(alg)
    alg_pickel = pickle.loads(alg_var)

    dump(alg_pickel ,os.path.join(path,f"Models/{symbol}.joblib"))
    
    
    

def lin_reg_sig(symbol):
    """ Function for predict the value of tommorow using ARIMA model"""
    
    # Create the weights if there is not in the folder
    try:
        alg = load(os.path.join(path,f"Models/{symbol}.joblib"))
    except:
        create_model_weights(symbol)
        alg = load(os.path.join(path,f"Models/{symbol}.joblib"))
    
    # Take the lastest percentage of change 
    data = MT5.get_data(symbol, 3500)[["close"]].pct_change(1)
    data["close t-1"] = data[["close"]].shift(1)
    X = data["close t-1"].iloc[-1].reshape(-1,1)
    
    
    # Find the signal
    prediction = alg.predict(X)
    long = prediction[0][0] > 0
    short=not long
    
    
    return long, short




Mode = "YES" #input("Live Trading (YES) or Screener (NO): ").upper()


if Mode=="YES":
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")




info_order = {
    "Nasdaq 100": ["NAS100", 0.1],
    "Russel 2000": ["US2000", 0.1],
    "Gold USD": ["XAUUSD", 0.01],
    "S&P 500": ["US500", 0.1],
    "Us dollar vs Canadian dollar": ["USDCAD", 0.01],
    "Euro vs USdollar": ["EURUSD", 0.01]
}


start = datetime.now().strftime("%H:%M:%S")
while True:
    # Verfication for launch
    if datetime.now().weekday() not in (5,1):
        is_time = datetime.now().strftime("%H:%M:%S") == start #"23:59:59"
    else:
        is_time = False

    
    # Launch the algorithm
    if is_time:

        # Open the trades
        for asset in info_order.keys():

            # Initialize the inputs
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Create the signals
            long, short = lin_reg_sig(symbol)

             # Run the algorithm
            if Mode=="YES":
                MT5.run(symbol, long, short,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {long}\t"
                     f"Sell: {short}")
    time.sleep(1)

------------------------------------------------------------------
Date:  2021-09-12 10:44:34
Balance: 974.68 USD, 	Equity: 974.68 USD, 	Profit: 0.0 USD
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-09-12 10:44:34
SYMBOL: NAS100
BUY: True 	  SHORT: False
POSITION: None 	 ID: None
OPEN LONG TRADE: Market closed
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-09-12 10:44:34
SYMBOL: US2000
BUY: True 	  SHORT: False
POSITION: None 	 ID: None
OPEN LONG TRADE: Market closed
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-09-12 10:44:34
SYMBOL: XAUUSD
BUY: True 	  SHORT: False
POSITION: None 	 ID: None
OPEN LONG TRADE: Market closed
------------------------------------------------------------------
---

In [4]:
import warnings
warnings.filterwarnings("ignore")
from MT5 import *
import numpy as np
import ta
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
import time
import pickle
from joblib import dump, load

path = "" # Ex: C:/Desktop/Python_for_finance_and_algorithmic_trading/ChapterN/Models

def create_model_weights(symbol):
    """ Weights for Linear regression on the percentage change"""
    # Import the data
    data = MT5.get_data(symbol, 3500)[["close"]].pct_change(1)
    
    # Create new variable
    data["close t-1"] = data[["close"]].shift(1)
    
    # Split the data
    data = data.dropna()
    split = int(0.80*len(data))
    
    # Train set creation
    X_train = data[["close t-1"]].iloc[:split]
    y_train = data[["close"]].iloc[:split]

    # Test set creation
    X_test = data[["close t-1"]].iloc[split:]
    
    # Create the model
    alg = LinearRegression()

    # Fit the model
    alg.fit(X_train, y_train)
    
    # Save the model
    alg_var = pickle.dumps(alg)
    alg_pickel = pickle.loads(alg_var)

    dump(alg_pickel ,f"Models/{symbol}.joblib")
    

create_model_weights("EURUSD")